In [34]:
# Import required libraries
import pandas as pd
import random


# Data Loading Function

In [35]:
def load_pokemon_data():
    # Load the Pokemon dataset
    df = pd.read_csv('deteset/pokemon.csv')
    # Create evolution chains dictionary
    evolution_chains = {}
    
    # Group Pokemon by their base stats and types to identify evolution chains
    for idx, row in df.iterrows():
        base_form = row['name']
        base_total = row['base_total']
        
        # Look for potential evolutions (Pokemon with similar types and higher stats)
        evolutions = df[
            (df['type1'] == row['type1']) & 
            (df['base_total'] > base_total) &
            (abs(df['pokedex_number'] - row['pokedex_number']) <= 2)  # Usually evolutions are close in Pokedex
        ]['name'].tolist()
        
        if evolutions:
            evolution_chains[base_form] = evolutions
    
    return df, evolution_chains

# Pokemon Encounter Function

In [36]:
def encounter_pokemon(pokemon_data):
    # Randomly select a Pokemon (excluding legendaries)
    non_legendary = pokemon_data[pokemon_data['is_legendary'] == 0]
    pokemon = non_legendary.sample(1).iloc[0]
    
    # Simulate random stats (IVs)
    iv_hp = random.randint(0, 31)
    iv_attack = random.randint(0, 31)
    iv_defense = random.randint(0, 31)
    iv_speed = random.randint(0, 31)
    
    # Calculate total IVs
    total_ivs = iv_hp + iv_attack + iv_defense + iv_speed
    iv_percentage = (total_ivs / 124) * 100  # 124 is max possible IV total
    
    return pokemon, iv_percentage

# Evolution Evaluation Function

In [37]:
def evaluate_evolution(pokemon, iv_percentage, pokemon_data, evolution_chains):
    if pokemon['name'] not in evolution_chains:
        return f"This Pokemon ({pokemon['name']}) has no evolutions."
    
    evolutions = evolution_chains[pokemon['name']]
    base_stats = pokemon['base_total']
    
    evolution_info = []
    for evolution in evolutions:
        evolved_stats = pokemon_data[pokemon_data['name'] == evolution]['base_total'].iloc[0]
        stat_increase = evolved_stats - base_stats
        evolution_info.append((evolution, stat_increase))
    
    # Make recommendation based on IVs and potential stat gains
    if iv_percentage >= 85:
        recommendation = "HIGHLY RECOMMENDED"
    elif iv_percentage >= 70:
        recommendation = "Recommended"
    elif iv_percentage >= 50:
        recommendation = "Consider evolving"
    else:
        recommendation = "Not recommended - Low IVs"
    
    result = f"\nPokemon: {pokemon['name']}"
    result += f"\nIV Quality: {iv_percentage:.1f}%"
    result += f"\nEvolution Recommendation: {recommendation}"
    result += "\n\nPossible Evolutions:"
    
    for evolution, stat_increase in evolution_info:
        result += f"\n- {evolution} (Stats +{stat_increase})"
    
    return result

# Log Encounter Function

In [38]:
def log_encounter(pokemon, iv_percentage, evaluation):
    # Create a dictionary with the encounter data
    encounter_data = {
        'date': pd.Timestamp.now(),
        'pokemon': pokemon['name'],
        'iv_percentage': round(iv_percentage, 1),
        'evaluation': evaluation
    }

    # Add a separator row
    separator_data = {
        'date': '-' * 20,
        'pokemon': '-' * 20,
        'iv_percentage': '-' * 20,
        'evaluation': '-' * 20
    }
    
    # Convert to DataFrame
    encounter_df = pd.DataFrame([encounter_data, separator_data])
    
    # Append to CSV file (create if doesn't exist)
    encounter_df.to_csv('pokemon_encounters.csv', 
                       mode='a', 
                       header=not pd.io.common.file_exists('pokemon_encounters.csv'),
                       index=False)

# Interactive Cell (run this cell multiple times to simulate different encounters)

In [49]:
pokemon, iv_percentage = encounter_pokemon(pokemon_data)
evaluation = evaluate_evolution(pokemon, iv_percentage, pokemon_data, evolution_chains)
print(evaluation)
log_encounter(pokemon, iv_percentage, evaluation)


Pokemon: Kadabra
IV Quality: 85.5%
Evolution Recommendation: HIGHLY RECOMMENDED

Possible Evolutions:
- Alakazam (Stats +200)
